# Kalman and Bayesian Filter in python

## Chapter 1 gh filter workbook

https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/01-g-h-filter.ipynb

In [1]:
import numpy as np
import pandas as pd
import altair as alt


weights = [158.0, 164.2, 160.3, 159.9, 162.1, 164.6, 
           169.6, 167.4, 166.4, 171.0, 171.2, 172.6]

In [2]:
time_step = 1.0  # day
scale_factor = 4.0 / 10.0

In [3]:
def predict_using_gain_guess(estimated_weight, z, gain_rate):
    predicted_weight = estimated_weight + gain_rate * time_step
    estimated_weight = predicted_weight + scale_factor * (z - predicted_weight)
    return estimated_weight, predicted_weight

estimated_weight = 160.0  # init guess
df = pd.DataFrame({'estimated': [estimated_weight], 'z': [np.nan], 'predicted': [np.nan]})
for z in weights:
    estimated_weight, predicted_weight  = predict_using_gain_guess(
        estimated_weight, z, gain_rate = 1.0)
    df = pd.concat([df, pd.DataFrame(
        [[estimated_weight, z, predicted_weight]],
        columns=['estimated', 'z', 'predicted']
    )], ignore_index=True)

df = df.reset_index().rename(columns={'index': 'step'})
df.head()

,step,estimated,z,predicted
0,0,160.0000,NaN,NaN
1,1,159.8000,158.0,161.000
2,2,162.1600,164.2,160.800
3,3,162.0160,160.3,163.160
4,4,161.7696,159.9,163.016


In [4]:
df = pd.melt(df, id_vars='step')
df.head()

,step,variable,value
0,0,estimated,160.0000
1,1,estimated,159.8000
2,2,estimated,162.1600
3,3,estimated,162.0160
4,4,estimated,161.7696


In [5]:
alt.Chart(df).mark_line().encode(
    x='step',
    y=alt.Y('value', scale=alt.Scale(zero=False)),
    color='variable',
    shape=alt.Shape('variable')
)  #.interactive()
# does not render this chart in the github

alt.Chart(...)

## Chapter 2 Discrete Bayes Filter workbook

https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/02-Discrete-Bayes.ipynb

### Tracking a dog

* H: Hallway에서 dog의 위치
* D: door/wall
* S: Sensor signal door/wall

알고자 하는 사항 $P(H|S)$

* $P(H|S) \propto P(H) \times P(S|H)$

In [2]:
import numpy as np
import pandas as pd
import altair as alt

from filterpy.discrete_bayes import normalize

* z: measurement 1 door, 0 wall
* z_prob: measurement가 정확할 확률

In [3]:
def scaled_update(hallway, H_prior, S, S_prob):
    likelihood = np.array([
        S_prob if H == S else 1.0 - S_prob
        for H in hallway
    ])
    posterior = H_prior * likelihood
    normalize(posterior)
    return posterior

hallway = np.array([1, 1, 0, 0, 0, 0, 0, 0, 1, 0])
H_prior = np.array([0.1] * 10)

reading = 1  # 1 is 'door'
posterior = scaled_update(hallway, H_prior, reading, 0.75)
posterior, posterior.sum()

(array([0.1875, 0.1875, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
        0.1875, 0.0625]), 1.0)

In [4]:
alt.Chart(
    pd.DataFrame({'pos': range(10), 'prob': posterior})
).mark_bar().encode(
    x='pos',
    y='prob'
)

alt.Chart(...)